# Databricks Unity Catalog - Table ACL

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/uc-base-0.png?raw=true" style="float: right" width="500px"/> 

The main feature of Unity Catalog is to provide you an easy way to setup Table ACL (Access Control Level), but also build Dynamic Views based on each individual permission.

Typically, Analysts will only have access to customers from their country and won't be able to read GDPR/Sensitive informations (like email, firstname etc.)

A typical workflow in the Lakehouse architecture is the following:

* Data Engineers / Jobs can read and update the main data/schemas (ETL part)
* Data Scientists can read the final tables and update their features tables
* Data Analyst have READ access to the Data Engineering and Feature Tables and can ingest/transform additional data in a separate schema.
* Data is masked/anonymized dynamically based on each user access level

With Unity Catalog, your tables, users and groups are defined at the account level, cross workspaces. Ideal to deploy and operate a Lakehouse Platform across all your teams.

Let's see how this can be done with the Unity Catalog

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=governance&org_id=3759185753378633&notebook=%2F00-UC-Table-ACL&demo_name=uc-01-acl&event=VIEW&path=%2F_dbdemos%2Fgovernance%2Fuc-01-acl%2F00-UC-Table-ACL&version=1">

### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-uc-01-acl-maynard` from the dropdown menu ([open cluster configuration](https://adb-3759185753378633.13.azuredatabricks.net/#setting/clusters/0601-075249-kvumx2hn/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('uc-01-acl')` or re-install the demo: `dbdemos.install('uc-01-acl')`*

## Cluster setup for UC

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/clusters_assigned.png?raw=true" style="float: right" width="500px"/>


To be able to run this demo, make sure you create a cluster with the security mode enabled.

Go in the compute page, create a new cluster.

Under "Access mode", select "Single User" and your UC-user (the user needs to exist at the workspace and the account level)

In [0]:
%run ./_resources/00-setup

## Creating the CATALOG

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/uc-base-1.png?raw=true" style="float: right" width="800px"/> 

The first step is to create a new catalog.

Unity Catalog works with 3 layers:

* CATALOG
* SCHEMA (or DATABASE)
* TABLE

To access one table, you can specify the full path: `SELECT * FROM &lt;CATALOG&gt;.&lt;SCHEMA&gt;.&lt;TABLE&gt;`

Note that the tables created before Unity Catalog are saved under the catalog named `hive_metastore`. Unity Catalog features are not available for this catalog.

Note that Unity Catalog comes in addition to your existing data, not hard change required!

In [0]:
%python
#The demo will create and use the catalog defined:
# see the catalog value in the ./config file
spark.sql(f'CREATE CATALOG IF NOT EXISTS {catalog}');
#Make it default for future usage (we won't have to specify it)
spark.sql(f'USE CATALOG main');

In [0]:
-- the catalog has been created for your user and is defined as default. All shares will be created inside.
-- make sure you run the 00-setup cell above to init the catalog to your user. 
SELECT CURRENT_CATALOG();

## Creating the SCHEMA
Next, we need to create the SCHEMA (or DATABASE).

Unity catalog provide the standard GRANT SQL syntax. We'll use it to GRANT CREATE and USAGE on our SCHEMA to all the users for this demo.

They'll be able to create extra table into this schema.

In [0]:
%python
# see schema value in the ./config file
spark.sql(f'CREATE SCHEMA IF NOT EXISTS {schema}');
spark.sql(f'USE SCHEMA {schema}');

## Creating our table

We're all set! We can use standard SQL to create our tables.

We'll use a customers dataset, loading data about users (id, email etc...)

Because we want our demo to be available for all, we'll grant full privilege to the table to all USERS.

Note that the table owner is the current user. Owners have full permissions.<br/>
If you want to change the owner you can set it as following: ```ALTER TABLE <catalog>.uc_acl.customers OWNER TO `account users`;```

In [0]:
CREATE TABLE IF NOT EXISTS customers (
  id STRING,
  creation_date STRING,
  firstname STRING,
  lastname STRING,
  country STRING,
  email STRING,
  address STRING,
  gender DOUBLE,
  age_group DOUBLE);
-- GRANT SELECT, MODIFY on TABLE customers TO `account users`;  -- for the demo only, allow all users to edit the table - don't do that in production!

## Our customer data was filled for us!

The initialization cell already filled the table for us with fake data for the demo, let's review it's content.

In [0]:
SELECT * FROM  customers

## Granting users or group access

Let's now use Unity Catalog to GRANT permission on the table.

Unity catalog let you GRANT standard SQL permission to your objects, using the Unity Catalog users or group:

### Creating groups

Databricks groups can be created at the account level using the Account Admin UI, or the SCIM API. Here, we created the `dataengineers` group for this demo.

*Note on workspace-level groups: you can also create groups at a workspace level, however, we recommend managing permissions with UC at an account level.*

In [0]:
-- Let's grant all users a SELECT
-- GRANT SELECT ON TABLE customers TO `account users`; -- skip it for the demo, uncomment to make it available to all users!

-- We'll grant an extra MODIFY to our Data Engineer
-- Note: make sure you created the dataengineers group first as an account admin!
GRANT SELECT, MODIFY ON TABLE customers TO `dataengineers`;

In [0]:
SHOW GRANTS ON TABLE customers

## Conclusion

Unity Catalog gives you Table ACL permissions, leveraging users, group and table across multiple workspaces.

But UC not only gives you control over Tables. You can do more advanced permission and data access pattern such as dynamic masking at the row level.

### Next: Fine Grain Access control

Databricks Unity Catalog provides built-in capabilities to add dynamic masking on columns or rows.

Let's see how this can be done in the [01-Row-Column-access-control notebook ]($./01-Row-Column-access-control).